## Overview

This is a project for automating trading. I am using Twelve Data API to access stock data. My algorithms will probably rely on technical analysis rather than fundamental analysis: data about the stocks movements rather than company releases, as I can automate this stuff easily.

## TwelveData API Tools

[Here is the documentation page for the API.](https://twelvedata.com/docs)

[Here is a tutorial for a library.](https://twelvedata.com/blog/get-high-quality-financial-data-directly-into-python)

[Here is the library's GitHub repository.](https://github.com/twelvedata/twelvedata-python)


I'm allowed up to 800 calls per day.


## Setup

The first steps in this project are setting up this venv with my desired libraries. I am installing: 

>`pip install twelvedata[matplotlib, plotly]`


## First Steps

I will begin by practicing fetching data from the API and displaying it. This will help me using tools to graphically represent data.

In [2]:
from twelvedata import TDClient
import json
import datetime


td = TDClient(apikey="a2edbe9124ce46adba66a0b007253f80")

# Construct the necessary time series
ts = td.time_series(
    symbol="AAPL",
    interval="1min",
    timezone="America/New_York",
    outputsize=1
).as_json()

# with open("results.txt", "w") as f: 
#     f.write(ts.to_string())

with open("results.txt", "w") as f: 
    f.write(json.dumps(ts, indent=4))


## First Algorithm

This algorithm will buy some AAPL stock if the price is below a certain number. 

First, I will create the sandbox that keeps track of my possesions. I will have the API connection to the external world as a given. Second, I will work on using the API to interact with the world to get the data I want.

As I go forward, I will iterate over the first step to improve the algorithm. 

Stipulations:
* 2023-04-04 8:11 AM: The API returns a range of prices of an asset over a time interval. I will take the current price to be the open.

In [10]:
"""
This is the basic structure for the trading algorithm. This creates the environment that manages possessions and transactions. That occurs client-side. 


The API is contained in the function that fetches data. Everything else occus on the machine.
"""
from twelvedata import TDClient
import json

td = TDClient(apikey="a2edbe9124ce46adba66a0b007253f80")

# Bridge to the stock market.
def fetch_stock_data(stock_ticker, interval, size):
    """
    Gets the necessary data from the API as a PANDAS dataframe.

    Argument:
        stock_ticker (string): ticker 
        interval (string): time frame (1min, 5min, 15min, 30min, 45min, 1h, 2h, 4h, 8h, 1day, 1week, 1month)
        size (int): the number of data points to retrieve

    Return (dict):
        price per share at time of call
    """
    # Construct the time series
    ts = td.time_series(
        symbol=stock_ticker,
        interval=interval,
        timezone="America/New_York",
        outputsize=size
    ).as_pandas()
    return ts

base = fetch_stock_data("AAPL", "1h", 1)
print(base)


                       open   high        low   close   volume
datetime                                                      
2023-09-21 11:30:00  174.92  175.7  174.83501  175.17  7364408


### This will define the decision algorithms

In [18]:
# Buy/Sell if stock is increasing / decreasing in value 
def decide_buy(stock_data):
    """
    Decide whether to buy a stock
    Buys if the 50-day moving average is greater than the 100-day moving average

    Argument: 
        stock_data (pandas df)

    Return:
        choice (boolean)
    """
    long = stock_data['close'].copy().ewm(span=100).mean()
    short = stock_data['close'].copy().ewm(span=50).mean()
    return short[0] > long[0] and abs(short - long) < 1

def decide_sell(stock_data):
    """
    Decide whether to sell a stock

    Argument: 
        stock_data (pandas df)

    Return:
        choice (boolean)
    """
    long = stock_data['close'].copy().ewm(span=100).mean()
    short = stock_data['close'].copy().ewm(span=50).mean()
    return short[0] < long[0] and abs(short - long) < 1



In [3]:
import pandas as pd


long_is_over_short = True

stock_data = fetch_stock_data("AAPL", "1day", 300)

print(len(stock_data))

long = stock_data['close'].copy().rolling(100).mean()
short = stock_data['close'].copy().rolling(50).mean()

long = long[long.notnull()]
short = short[short.notnull()]

for i in range(300): 
    print(long.iloc[0])
    print("--")
    print(short.iloc[0])

    if short.iloc[0] > long.iloc[0] and long_is_over_short:
        long_is_over_short = not long_is_over_short
        print("change")
    else: print("nothing")

    print("===")



300
181.73460060000002
--
184.256701
change
===
181.73460060000002
--
184.256701
nothing
===
181.73460060000002
--
184.256701
nothing
===
181.73460060000002
--
184.256701
nothing
===
181.73460060000002
--
184.256701
nothing
===
181.73460060000002
--
184.256701
nothing
===
181.73460060000002
--
184.256701
nothing
===
181.73460060000002
--
184.256701
nothing
===
181.73460060000002
--
184.256701
nothing
===
181.73460060000002
--
184.256701
nothing
===
181.73460060000002
--
184.256701
nothing
===
181.73460060000002
--
184.256701
nothing
===
181.73460060000002
--
184.256701
nothing
===
181.73460060000002
--
184.256701
nothing
===
181.73460060000002
--
184.256701
nothing
===
181.73460060000002
--
184.256701
nothing
===
181.73460060000002
--
184.256701
nothing
===
181.73460060000002
--
184.256701
nothing
===
181.73460060000002
--
184.256701
nothing
===
181.73460060000002
--
184.256701
nothing
===
181.73460060000002
--
184.256701
nothing
===
181.73460060000002
--
184.256701
nothing
===
181.734

In [8]:
stock_data = fetch_stock_data("AAPL", "1day", 100)

long = stock_data['close'].copy().rolling(100).mean()
short = stock_data['close'].copy().rolling(50).mean()

long.head()
long = long[::-1]
print("--")
long.head()


--


datetime
2023-04-27    181.736801
2023-04-28           NaN
2023-05-01           NaN
2023-05-02           NaN
2023-05-03           NaN
Name: close, dtype: float64

In [3]:
# Figuring out how rolling windows work

import pandas as pd


base = pd.DataFrame([1, 2, 3, 4, 5])

rllng_avg2 = base.copy().rolling(2).mean()
rllng_avg3 = base.copy().rolling(3).mean()
rllng_avg4 = base.copy().rolling(4).mean()
rllng_avg5 = base.copy().rolling(5).mean()

print(rllng_avg2)
print("-")

print(rllng_avg3)
print("-")

print(rllng_avg4)
print("-")

print(rllng_avg5)
print("-")

     0
0  NaN
1  1.5
2  2.5
3  3.5
4  4.5
-
     0
0  NaN
1  NaN
2  2.0
3  3.0
4  4.0
-
     0
0  NaN
1  NaN
2  NaN
3  2.5
4  3.5
-
     0
0  NaN
1  NaN
2  NaN
3  NaN
4  3.0
-


In [9]:
# Figuring out how to search a list for a dict with certain condition

base = [{"a": 1}, {"a": 2}, {"a": 3}]

def func(variable):
    return variable["a"] == 1

obj = list(filter(func, base))[0]
print(obj)

{'a': 1}


### Investigating whether moving average heuristic is profitable with simulation:

In [6]:
import pandas as pd
import random

# Decision mechanics

def decide_buy(series: pd.DataFrame) -> bool:
    return short_average(series) > long_average(series)

def decide_sell(series: pd.DataFrame) -> bool:
    return short_average(series) < long_average(series)

def long_average(series: pd.DataFrame) -> float:
    raw = series.copy().rolling(5).mean()
    return raw.iloc[-1]

def short_average(series: pd.DataFrame) -> float:
    raw = series.copy().rolling(3).mean()
    print(raw)
    return raw.iloc[-1]


# Price series

prices = pd.DataFrame(
    [random.randint(50, 150) for i in range(105)]
)


# Initialize variables 

funds = 500
shares = 0 
long_over_short = long_average(prices[0:4]) > short_average(prices[0:4])


# Simulation

for i in range(100):
    i+=4
    if decide_buy(prices[4:i]):
        funds -= prices.iloc[i]
        shares += 1
        long_over_short = not long_over_short
    elif decide_sell(prices[4:i]):
        funds += prices.iloc[i]
        shares -= 1
        long_over_short = not long_over_short

# Final tally

print(f"Final balance: {funds + shares * prices.iloc[-1]}")
print(f"Uninvested cash: {funds}")
print(f"{shares} shares @ {prices.iloc[-1]} = {funds + shares * prices.iloc[-1]} in stock")


            0
0         NaN
1         NaN
2   83.666667
3  105.666667
Empty DataFrame
Columns: [0]
Index: []


IndexError: single positional indexer is out-of-bounds

In [3]:
import pandas as pd

df = pd.DataFrame([1, 2, 3, 4, 5])
print(df.iloc[-1])

0    5
Name: 4, dtype: int64
